In [10]:
# import statement and loading environment variables

import os
import pathlib
from google.cloud import storage 
import json
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [21]:
# User set variables

user = '5037547138'


bot_name = 'wilke'
# TODO - Can be used to start with a baseline prompt, then customize the bot on creation with the prompt. 

# prompt = TODO - add prompt, allowing the customerization of the bot on creation

In [22]:
# Standard variables and formulas for GCP

working_dir = pathlib.Path.cwd()
bucket_name = 'gpt_chat_logs'
chat_file_folder = working_dir.joinpath('chat_logs')
file_path = chat_file_folder.joinpath('{}_{}_chat.json'.format(user, bot_name))
file_name = ('{}_{}_chat.json'.format(user, bot_name))
STORAGE_CLASSES = ('STANDARD', 'NEARLINE', 'COLDLINE', 'ARCHIVE')
storage_client = storage.Client()

In [23]:
# Class for working with Google Cloud Storage

class GCStorage:
    def __init__(self, storage_client):
        self.client = storage_client

    # Create a bucket
    def create_bucket(self, bucket_name, storage_class, bucket_location='US'):
        bucket = self.client.bucket(bucket_name)
        bucket.storage_class = storage_class
        return self.client.create_bucket(bucket, bucket_location)        

    # Get a bucket
    def get_bucket(self, bucket_name):
        return self.client.get_bucket(bucket_name)

    # List all buckets
    def list_buckets(self):
        buckets = self.client.list_buckets()
        return [bucket.name for bucket in buckets]

    # Upload a file to a bucket
    def upload_file(self, bucket, blob_destination, file_path):
        blob = bucket.blob(blob_destination)
        blob.upload_from_filename(file_path, content_type='application/json')
        return blob

    # List all blobs in a bucket
    def list_blobs(self, bucket_name):
        return self.client.list_blobs(bucket_name)
    
    def download_blob(self, bucket_name, blob_name, file_path):
        bucket = self.client.get_bucket(bucket_name)
        blob = bucket.blob(blob_name)
        blob.download_to_filename(file_path)
        return blob
    
    
    def load_file(self, bucket, blob_name):
        bucket = storage_client.get_bucket(bucket)
        blob = bucket.blob(blob_name)
        data = json.loads(blob.download_as_string(client=None))
        return data

In [24]:
# Initializing the GCP storage class and loading the chat log file
gcs = GCStorage(storage_client)
CHAT_LOG_FILE = gcs.load_file(bucket_name, file_name)
chat_log = CHAT_LOG_FILE


In [25]:
# Create a bucket if it doesn't exist and chat log management functions

def get_or_create_bucket(bucket_name):
    if not bucket_name in gcs.list_buckets():
        gcs.create_bucket(bucket_name, STORAGE_CLASSES[0])
    else:
        gcs.get_bucket(bucket_name)
        
        
# Create a new chat log file in the local chat_logs folder
def create_chat_log_file(user, bot_name):
    # TODO This function requires me to manually load the starting json 
    # Opportunity to integrate the startup exeperience while solving this manual operation
    filename = f'{user}_{bot_name}_chat.json'
    file_path = chat_file_folder.joinpath(filename)
    with open(file_path, 'w') as f:
        f.write('')
    return file_path

# upload the chat log file to the bucket
def upload_chat_log_file(bucket_name, file_path):
    name = f'{user}_{bot_name}_chat.json'
    filename = name
    bucket = gcs.get_bucket(bucket_name)
    blob = gcs.upload_file(bucket, filename, file_path)
    return blob

In [26]:
#=========================================================
## Called functions that will work correctly

# create_chat_log_file(user, bot_name)
# upload_chat_log_file(bucket_name, file_path)
# data = gcs.load_file(bucket_name, file_name)
# print(data  )

#=========================================================

In [27]:
#=========================================================
# Test function to make sure the model is returning a response

# completion = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo",
#   messages = [
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": "Who won the world series in 2020?"},
#             {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
#             {"role": "user", "content": "Where was it played?"}
#              ] 
#             )

# print(completion.choices[0].message)
#=========================================================

In [28]:
# Function to handle the chat log and return the response

def text(question):
    bucket = gcs.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",


        messages = chat_log)
    question = str(question)
    return_response = (response)['choices'][0]['message']['content']
    
    new_log = chat_log
    new_log.append({"role": "user", "content": question})
    new_log.append({"role": "assistant", "content": return_response})
    
    json_data = json.dumps(new_log, indent=4)
    
    # print(json_data)
    # print(new_log)

    blob.upload_from_string(json_data, content_type='application/json')

    return return_response

In [30]:
# Test Question
text('Are you going to finally work?')

'I apologize if I gave you the impression that I am not working. I am functioning properly and ready to assist you with any questions or tasks you have. How may I assist you today?'

In [ ]:
# === End

In [ ]:
# Text for conversation generation